# Case Study: Modelling obesity for inhabitants of Utrecht by making use of ABM

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import itertools
import datetime
import random
import campo
import math
import os

from shapely.geometry import Polygon, Point

import pcraster.framework as pcrfw
import pcraster as pcr

from osgeo import gdal, ogr, osr

import lue.data_model as ldm

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the datasets

In [2]:
# Reading pop dataframe
tot_pop = pd.read_csv("tot_pop.csv", delimiter = ';')
# Selecting just neccesary columns
tot_pop = tot_pop[["Wijken", "totaal aantal inwoners|2012"]]
# Calculating share of total population per district
tot_pop["%pop"] = tot_pop["totaal aantal inwoners|2012"] / tot_pop["totaal aantal inwoners|2012"].sum()
# Obtaining the number of agents per district
agents_n = 20000
#Getting number of agents to select per district
tot_pop["number_agents"] = round(tot_pop["%pop"] * agents_n, 0).astype(int)
display(tot_pop)

,Wijken,totaal aantal inwoners|2012,%pop,number_agents
0,West,27351,0.086478,1730
1,Noordwest,41219,0.130326,2607
2,Overvecht,31274,0.098882,1978
3,Noordoost,36544,0.115544,2311
4,Oost,30692,0.097042,1941
5,Binnenstad,16813,0.053159,1063
6,Zuid,26304,0.083168,1663
7,Zuidwest,36035,0.113935,2279
8,Leidsche Rijn,26708,0.084445,1689
9,Vleuten-De Meern,43337,0.137022,2740


In [3]:
# Read information about percentage of inhabitants with overweight in each neighbourhood
overweight = pd.read_csv('overweight_data.csv', sep = ';')
display(overweight)

,Wijken,"overgewicht (inclusief obesitas), 18+ jaar|2012","overgewicht (inclusief obesitas), 18+ jaar|2014","overgewicht (inclusief obesitas), 18+ jaar|2016","overgewicht (inclusief obesitas), 18+ jaar|2018","overgewicht (inclusief obesitas), 18+ jaar|2020"
0,West,37,33,29,30,31
1,Noordwest,40,38,37,32,39
2,Overvecht,48,49,49,51,50
3,Noordoost,31,32,35,30,27
4,Oost,24,24,19,24,24
5,Binnenstad,25,23,22,32,25
6,Zuid,38,38,32,34,38
7,Zuidwest,40,38,46,35,39
8,Leidsche Rijn,42,41,40,39,42
9,Vleuten-De Meern,53,48,47,45,52


In [4]:
#Merge df of number of agents an % of overweight
tot_pop = tot_pop.merge(overweight, on = 'Wijken')
display(tot_pop)

,Wijken,totaal aantal inwoners|2012,%pop,number_agents,"overgewicht (inclusief obesitas), 18+ jaar|2012","overgewicht (inclusief obesitas), 18+ jaar|2014","overgewicht (inclusief obesitas), 18+ jaar|2016","overgewicht (inclusief obesitas), 18+ jaar|2018","overgewicht (inclusief obesitas), 18+ jaar|2020"
0,West,27351,0.086478,1730,37,33,29,30,31
1,Noordwest,41219,0.130326,2607,40,38,37,32,39
2,Overvecht,31274,0.098882,1978,48,49,49,51,50
3,Noordoost,36544,0.115544,2311,31,32,35,30,27
4,Oost,30692,0.097042,1941,24,24,19,24,24
5,Binnenstad,16813,0.053159,1063,25,23,22,32,25
6,Zuid,26304,0.083168,1663,38,38,32,34,38
7,Zuidwest,36035,0.113935,2279,40,38,46,35,39
8,Leidsche Rijn,26708,0.084445,1689,42,41,40,39,42
9,Vleuten-De Meern,43337,0.137022,2740,53,48,47,45,52


In [5]:
# Read information about average income in each neighbourhood
income = pd.read_csv('income_data.csv', sep = ',')
display(income)

,Wijken,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,West,33.7,34.6,38.3,39.0,40.8,41.4,43.3,45.9,47.6
1,Noordwest,28.4,29.1,32.0,32.6,33.9,34.8,36.4,38.9,41.1
2,Overvecht,25.6,25.6,27.6,28.2,29.1,29.7,30.7,32.4,34.0
3,Noordoost,39.7,40.4,44.9,45.8,47.3,47.8,49.2,51.9,52.8
4,Oost,38.9,39.6,44.4,43.2,44.9,45.6,47.2,50.6,50.7
5,Binnenstad,33.8,35.0,39.5,41.1,41.8,41.9,41.7,44.8,46.2
6,Zuid,31.8,32.3,35.3,36.1,37.2,37.5,38.7,40.6,42.3
7,Zuidwest,30.1,30.6,33.4,33.7,35.0,35.6,36.9,38.8,41.0
8,Leidsche Rijn,41.1,42.2,47.5,48.0,50.0,51.6,52.3,55.1,57.0
9,Vleuten-De Meern,42.2,43.2,49.4,49.3,51.8,53.6,55.3,60.2,60.9


In [6]:
# Read information about percentage of increase in income by neighbourhood
inc_income = pd.read_csv('income_data_increase.csv', sep = ',')

# Change the name of the columns that we need
inc_income.rename(columns = {'Unnamed: 1': 'increment_percentage'}, inplace=True)

# Divide the annual increment by 3 (because our timesteps are every three months)
inc_income['increment_percentage'] = inc_income['increment_percentage'].apply(lambda row: row/3)

display(inc_income)

,Wijken,increment_percentage
0,West,0.014000
1,Noordwest,0.015000
2,Overvecht,0.011667
3,Noordoost,0.011667
4,Oost,0.013000
5,Binnenstad,0.013000
6,Zuid,0.011667
7,Zuidwest,0.012667
8,Leidsche Rijn,0.013333
9,Vleuten-De Meern,0.015000


In [7]:
#Merge df with income data
tot_pop = tot_pop.merge(income, on = 'Wijken')
display(tot_pop)

,Wijken,totaal aantal inwoners|2012,%pop,number_agents,"overgewicht (inclusief obesitas), 18+ jaar|2012","overgewicht (inclusief obesitas), 18+ jaar|2014","overgewicht (inclusief obesitas), 18+ jaar|2016","overgewicht (inclusief obesitas), 18+ jaar|2018","overgewicht (inclusief obesitas), 18+ jaar|2020",2012,2013,2014,2015,2016,2017,2018,2019,2020
0,West,27351,0.086478,1730,37,33,29,30,31,33.7,34.6,38.3,39.0,40.8,41.4,43.3,45.9,47.6
1,Noordwest,41219,0.130326,2607,40,38,37,32,39,28.4,29.1,32.0,32.6,33.9,34.8,36.4,38.9,41.1
2,Overvecht,31274,0.098882,1978,48,49,49,51,50,25.6,25.6,27.6,28.2,29.1,29.7,30.7,32.4,34.0
3,Noordoost,36544,0.115544,2311,31,32,35,30,27,39.7,40.4,44.9,45.8,47.3,47.8,49.2,51.9,52.8
4,Oost,30692,0.097042,1941,24,24,19,24,24,38.9,39.6,44.4,43.2,44.9,45.6,47.2,50.6,50.7
5,Binnenstad,16813,0.053159,1063,25,23,22,32,25,33.8,35.0,39.5,41.1,41.8,41.9,41.7,44.8,46.2
6,Zuid,26304,0.083168,1663,38,38,32,34,38,31.8,32.3,35.3,36.1,37.2,37.5,38.7,40.6,42.3
7,Zuidwest,36035,0.113935,2279,40,38,46,35,39,30.1,30.6,33.4,33.7,35.0,35.6,36.9,38.8,41.0
8,Leidsche Rijn,26708,0.084445,1689,42,41,40,39,42,41.1,42.2,47.5,48.0,50.0,51.6,52.3,55.1,57.0
9,Vleuten-De Meern,43337,0.137022,2740,53,48,47,45,52,42.2,43.2,49.4,49.3,51.8,53.6,55.3,60.2,60.9


# Extracting Utrecht Houses Data

In [8]:
# Loading houses data
filename = "FINAL_HOUSES.gpkg"
data_source = ogr.GetDriverByName('GPKG').Open(filename, update=0)
houses = data_source.GetLayerByName('FINAL_HOUSES')

# Number of features in the layer (number of rows in the attribute table)
print("Number of features in the layer: {}".format(houses.GetFeatureCount()))

# To query these fields you first need to get the feature definition of the layer
locations_def = houses.GetLayerDefn()

# The number of fields (number of columns in a table) can be retrieved with GetFieldCount()
print("Number of columns: {}".format(locations_def.GetFieldCount()))

# Variables to store all points
loc_x = []
loc_y = []
loc_d = []

# Start cycle in houses
for i in range(1, houses.GetFeatureCount() + 1):
    # Features id start from 1
    feature =  houses.GetFeature(i)
    if feature != None:
        # Getting house geometry 
        house_geometry = feature.GetGeometryRef()
        # Getting house centroid for each feature
        centroid = house_geometry.Centroid()
        # Getting house coordinate x and appendin to list
        x = round(centroid.GetX(),1)
        loc_x.append(x)
        # Getting house coordinate y
        y = round(centroid.GetY(),1)
        loc_y.append(y)
        # Getting house neightboorhood
        name = feature.Wijknaam
        loc_d.append(name)

# To df
df_agent_loc = pd.DataFrame()
df_agent_loc["x"] = loc_x 
df_agent_loc["y"] = loc_y
df_agent_loc["district"] = loc_d 

display(df_agent_loc)

Number of features in the layer: 163621
Number of columns: 373


,x,y,district
0,136851.6,455611.6,Binnenstad
1,137077.7,455205.7,Binnenstad
2,134984.9,457267.4,Noordwest
3,134951.1,457349.4,Noordwest
4,131248.3,457115.1,Leidsche Rijn
...,...,...,...
160353,134649.2,454677.2,Zuidwest
160354,134646.9,454678.0,Zuidwest
160355,134649.1,454677.2,Zuidwest
160356,134648.2,454677.6,Zuidwest


In [9]:
#Format repair (name of the district is different)
tot_pop.loc[9,"Wijken"] = "Vleuten - De Meern"
display(df_agent_loc.groupby('district').size())

district
Binnenstad            13009
Leidsche Rijn         20599
Noordoost             19211
Noordwest             22523
Oost                  16143
Overvecht             19198
Vleuten - De Meern      121
West                  14834
Zuid                  14198
Zuidwest              20522
dtype: int64

In [10]:
np.random.seed(123)
agents_per_district = df_agent_loc.groupby('district').apply(lambda x: x.sample(tot_pop[tot_pop['Wijken'] == x.name]['number_agents'].values[0], replace = True)).reset_index(drop = True)
agents_per_district

,x,y,district
0,137137.6,456000.5,Binnenstad
1,135836.6,455877.5,Binnenstad
2,136757.0,455242.5,Binnenstad
3,136754.9,456546.1,Binnenstad
4,136634.1,456232.9,Binnenstad
...,...,...,...
19996,136000.5,454697.4,Zuidwest
19997,136250.8,453965.5,Zuidwest
19998,136337.3,454676.7,Zuidwest
19999,134468.3,454543.0,Zuidwest


In [11]:
# Check that we have the correct number of samples per neighbourhood
agents_per_district.groupby('district').count()

,x,y
district,,
Binnenstad,1063,1063
Leidsche Rijn,1689,1689
Noordoost,2311,2311
Noordwest,2607,2607
Oost,1941,1941
Overvecht,1978,1978
Vleuten - De Meern,2740,2740
West,1730,1730
Zuid,1663,1663


In [12]:
# Add a "duplicate" column to the data frame
agents_per_district['duplicate'] = agents_per_district.duplicated(subset = ['x', 'y'], keep = False)

In [13]:
# Define a function to add noise to latitude and longitude values that are duplicated
def add_noise(row):
    if row['duplicate'] == True:
        row['x'] += np.random.normal(loc = 0.0, scale = 0.001)
        row['y'] += np.random.normal(loc = 0.0, scale = 0.001)
    return row

# Apply the function to the data frame
np.random.seed(123)
agents_per_district = agents_per_district.apply(add_noise, axis = 1)

In [14]:
#Removing Vleuten de mern ("Not enought info")
agents_per_district = agents_per_district.loc[agents_per_district["district"]!= "Vleuten - De Meern"]
display(agents_per_district)

,x,y,district,duplicate
0,137137.600000,456000.500000,Binnenstad,False
1,135836.600000,455877.500000,Binnenstad,False
2,136756.998914,455242.500997,Binnenstad,True
3,136754.900000,456546.100000,Binnenstad,False
4,136634.100000,456232.900000,Binnenstad,False
...,...,...,...,...
19996,136000.500000,454697.400000,Zuidwest,False
19997,136250.800000,453965.500000,Zuidwest,False
19998,136337.300000,454676.700000,Zuidwest,False
19999,134468.300000,454543.000000,Zuidwest,False


In [15]:
# Check if we don't have duplicates anymore
display(agents_per_district[agents_per_district['duplicate'] == True])
agents_per_district['duplicate'] = agents_per_district.duplicated(subset=['x', 'y'], keep = False)
display(agents_per_district[agents_per_district['duplicate'] == True])
#Houses coords to csv
agents_per_district[["x","y"]].to_csv("new_agents.csv", index_label = None, index = False, header = False)

,x,y,district,duplicate
2,136756.998914,455242.500997,Binnenstad,True
13,136717.900283,455893.098494,Binnenstad,True
29,136756.999421,455242.501651,Binnenstad,True
36,136534.797573,455988.499571,Binnenstad,True
41,136595.801266,454789.999133,Binnenstad,True
...,...,...,...,...
19986,135923.800001,452873.100148,Zuidwest,True
19989,135305.899837,453596.100035,Zuidwest,True
19990,135855.899514,453316.297776,Zuidwest,True
19994,135903.700501,453077.401334,Zuidwest,True


,x,y,district,duplicate


### Generate values of BMI and income for the selected households

In [16]:
# Function to generate BMI values for each house in the neighbourhood based in the overweight % for that district
def create_BMI_column(group):
    np.random.seed(123)
    
    # Get the proportion of houses in which people are overweighted
    neighbourhood = tot_pop[tot_pop['Wijken'] == group.name]
    
    percentage = neighbourhood['overgewicht (inclusief obesitas), 18+ jaar|2012'].values[0]
    num_agents = neighbourhood['number_agents'].values[0]
    
    #Getting number to get in each side (normal or overweight side)
    overweighted = int(num_agents * (percentage / 100))
    normal = num_agents - overweighted
    
    #Setting low and high extremes of the distributions
    low = np.random.uniform(low = 18.5, high = 24.9, size = (normal, ))
    high = np.random.uniform(low = 25.0, high = 35.0, size = (overweighted, ))
    
    #Saving generated bmi
    group['BMI'] = np.append(low, high)
    
    return group

In [17]:
# Function to generate income values for each house in the neighbourhood
def create_income_column(group):
    np.random.seed(123)
    
    # Get the proportion of houses in which people are overweighted
    neighbourhood = tot_pop[tot_pop['Wijken'] == group.name]
    
    mean_value = neighbourhood['2012'].values[0] * 1000
    num_agents = neighbourhood['number_agents'].values[0]
    
    # Generate points with a normal distribution
    points = np.random.normal(mean_value, 3000, num_agents)

    # Clamp the values to be between min_income and max_value
    points = np.clip(points, 16000, 100000)
    
    group['annual_income'] = points
    
    return group

In [18]:
# Create new column BMI and income based on subgroup of 'district' column
agents_per_district = agents_per_district.groupby('district').apply(create_BMI_column)
agents_per_district = agents_per_district.groupby('district').apply(create_income_column)

agents_per_district = agents_per_district.merge(inc_income, left_on = 'district', right_on='Wijken')
display(agents_per_district)

,x,y,district,duplicate,BMI,annual_income,Wijken,increment_percentage
0,137137.600000,456000.500000,Binnenstad,False,22.957403,30543.108190,Binnenstad,0.013000
1,135836.600000,455877.500000,Binnenstad,False,20.331292,36792.036340,Binnenstad,0.013000
2,136756.998914,455242.500997,Binnenstad,False,19.951849,34648.935494,Binnenstad,0.013000
3,136754.900000,456546.100000,Binnenstad,False,22.028415,29281.115858,Binnenstad,0.013000
4,136634.100000,456232.900000,Binnenstad,False,23.104601,32064.199244,Binnenstad,0.013000
...,...,...,...,...,...,...,...,...
17256,136000.500000,454697.400000,Zuidwest,False,26.271288,29359.515094,Zuidwest,0.012667
17257,136250.800000,453965.500000,Zuidwest,False,30.096850,29003.975290,Zuidwest,0.012667
17258,136337.300000,454676.700000,Zuidwest,False,25.705301,36652.941107,Zuidwest,0.012667
17259,134468.300000,454543.000000,Zuidwest,False,32.098342,31048.584777,Zuidwest,0.012667


### Importing data from restaurants

In [19]:
# Loading houses data
filename = "fast_food_utrecht.gpkg"
data_source = ogr.GetDriverByName('GPKG').Open(filename, update=0)
houses = data_source.GetLayerByName('fast_food_utrecht')

# Number of features in the layer (number of rows in the attribute table)
print("Number of features in the layer: {}".format(houses.GetFeatureCount()))

# To query these fields you first need to get the feature definition of the layer
locations_def = houses.GetLayerDefn()

# The number of fields (number of columns in a table) can be retrieved with GetFieldCount()
print("Number of columns: {}".format(locations_def.GetFieldCount()))

# Variables to store all points
loc_x = []
loc_y = []

# Start cycle in houses
for i in range(1, houses.GetFeatureCount() + 1):
    # Features id start from 1
    feature =  houses.GetFeature(i)
    if feature != None:
        # Getting house geometry 
        house_geometry = feature.GetGeometryRef()
        # Getting house centroid for each feature
        centroid = house_geometry.Centroid()
        # Getting house coordinate x and appendin to list
        x = round(centroid.GetX(),1)
        loc_x.append(x)
        # Getting house coordinate y
        y = round(centroid.GetY(),1)
        loc_y.append(y)
       
#To df
df_agent_loc = pd.DataFrame()
df_agent_loc["x"] = loc_x 
df_agent_loc["y"] = loc_y

# Save coordinates in a csv file
df_agent_loc.to_csv("new_agents_stores.csv", index_label = None, index = False, header = False)

Number of features in the layer: 212
Number of columns: 67


### Get the distance to the nearest restaurant for each household

In [20]:
#Function to get distance
def get_distance(df1,df2):
    #Loop in all houses
    for i in range (len(df1)):
        x_coord_house = df1.loc[i, "x"] 
        y_coord_house = df1.loc[i, "y"]
        dis = float("inf")
        distances = []
        
        #Loop in all stores
        for n in range (len(df2)):
            x_coord_store = df2.loc[n, "x"] 
            y_coord_store = df2.loc[n, "y"]
            #Calculate distance
            distance_calculated = math.sqrt(((x_coord_house-x_coord_store)**2 + (y_coord_house-y_coord_store)**2))
            #Capture minimum distance
            if dis > distance_calculated:
                dis = distance_calculated   
        df1.loc[i,"minimum_distance"] = round(dis,1)
    return df1

In [21]:
#Calculating the distance to closest fast food store
#Reseting index of dfs to avoid problems
agents_per_district.reset_index(drop=True , inplace = True)
df_agent_loc.reset_index(drop=True , inplace = True)
houses = agents_per_district.copy()
faststores = df_agent_loc.copy()

#Getting minimum distances between houses and stores
agents_per_district = get_distance(houses, faststores)
display(agents_per_district)

,x,y,district,duplicate,BMI,annual_income,Wijken,increment_percentage,minimum_distance
0,137137.600000,456000.500000,Binnenstad,False,22.957403,30543.108190,Binnenstad,0.013000,125.1
1,135836.600000,455877.500000,Binnenstad,False,20.331292,36792.036340,Binnenstad,0.013000,181.4
2,136756.998914,455242.500997,Binnenstad,False,19.951849,34648.935494,Binnenstad,0.013000,145.3
3,136754.900000,456546.100000,Binnenstad,False,22.028415,29281.115858,Binnenstad,0.013000,125.9
4,136634.100000,456232.900000,Binnenstad,False,23.104601,32064.199244,Binnenstad,0.013000,13.5
...,...,...,...,...,...,...,...,...,...
17256,136000.500000,454697.400000,Zuidwest,False,26.271288,29359.515094,Zuidwest,0.012667,377.4
17257,136250.800000,453965.500000,Zuidwest,False,30.096850,29003.975290,Zuidwest,0.012667,133.0
17258,136337.300000,454676.700000,Zuidwest,False,25.705301,36652.941107,Zuidwest,0.012667,224.6
17259,134468.300000,454543.000000,Zuidwest,False,32.098342,31048.584777,Zuidwest,0.012667,518.3


In [22]:
# Save information on a csv file
agents_per_district.to_csv("agents_with_bmi_income.csv", index_label = None, index = False, header = False)

## Calibration

In [23]:
#Set some parameters of abm
seed = 123
k4 = 30 # The effect of taxes 
p = 0 # This value indicated the taxes on fast food
n_timesteps = 24 #N Time-step value (8 years in total)

In [24]:
#DEFINING ENVIROMENT
class FoodEnvironment(pcrfw.DynamicModel):
    def __init__(self):
        pcrfw.DynamicModel.__init__(self)
        # Framework requires a clone
        # set a dummy clone
        pcr.setclone(10, 20, 10, 0, 0)

        ##########################
        # Differential Equations #
        ##########################

    # Get probability that a household will visit a fast food outlet, given their distance from the outlet, income, and taxes applied.
    def get_exponential(self, distance, income):
        return 1 / (1 + campo.exp( ( - k1 * distance ) + ( k3 * income * (1 + p * k4) ) ) )
    
    # This equation represents the change in BMI per unit time. 
    # The equation takes into account the effect of distance, the effect of time on the BMI and the probability that a household will visit a fast food.
    def diffEqBMI(self, bmi, income, distance):
        return ( k1 * ( (1 / distance)  * self.get_exponential(distance, income) ) - (k2 * bmi))
    
    def diffEqIncome(self, percentage, income):
        return percentage * income

    def initial(self):
        self.foodenv = campo.Campo()

        ##############
        # Households #
        ##############

        # Create households phenomenon
        self.hh = self.foodenv.add_phenomenon('hh')
        self.hh.add_property_set('fd', 'new_agents.csv')

        # Set initial bmi of households
        bmi_values = agents_per_district["BMI"]
        self.hh.fd.bmi = bmi_values.to_numpy()

        # Set initial income of households
        income_values = agents_per_district["annual_income"]
        self.hh.fd.income = income_values.to_numpy()/3

        # Set distance to the nearest food outlet
        income_values = agents_per_district["minimum_distance"]
        self.hh.fd.min_distance = income_values.to_numpy()
        
        percentage_values = agents_per_district["increment_percentage"]
        self.hh.fd.increment_percentage = percentage_values.to_numpy()

        # Technical detail: set map projection
        self.hh.set_epsg(28992)

        ##############
        # Foodstores #
        ##############

        # Create foodstores phenomenon
        self.fs = self.foodenv.add_phenomenon('fs')

        # Add the frontdoor property set
        self.fs.add_property_set('fd', 'new_agents_stores.csv')

        # Technical detail
        self.fs.set_epsg(28992)

        # Set the duration (years) of one time step
        self.timestep = 0.333333

        # Create the output lue data set
        self.foodenv.create_dataset("food_environment.lue")

        # Create real time settings for lue
        date = datetime.date(2000, 1, 2)
        time = datetime.time(12, 34)
        start = datetime.datetime.combine(date, time)
        unit = campo.TimeUnit.month
        stepsize = 4
        self.foodenv.set_time(start, unit, stepsize, self.nrTimeSteps())

        # Technical detail
        self.hh.fd.bmi.is_dynamic = True
        self.hh.fd.income.is_dynamic = True

        # Write the lue dataset
        self.foodenv.write()

    def dynamic(self):
        # update household income
        self.hh.fd.income = self.hh.fd.income + self.timestep * (self.diffEqIncome(self.hh.fd.increment_percentage, self.hh.fd.income))
        
        # Update household propensity
        # update household propensity
        self.hh.fd.bmi = self.hh.fd.bmi + self.timestep * (self.diffEqBMI(self.hh.fd.bmi, self.hh.fd.income, self.hh.fd.min_distance))

        # Print run duration info
        self.foodenv.write(self.currentTimeStep())

### Calibration per district

In [25]:
#Getting optimum k1,k2 and k3 per district
districts = agents_per_district["district"].value_counts()

#Obtaining a list with districts name
districts = list(districts.index)

In [26]:
#Set grid for iteration of possible k1,k2 and k3 combinations
k1_list =  np.arange(5, 30, 5).tolist()
k2_list =  np.arange(.005, .02, .005).tolist()
k3_list =  np.arange(.005, .025, .005).tolist()

#Defining storage variables
k1_storage =[]
k2_storage =[]
k3_storage =[]
mae_storage =[]
district_storage =[]

#Triple loop i the possible k values (size k1*k2*k3)
for k1 in k1_list:
    for k2 in k2_list:
        for k3 in k3_list:
            
            #Running ABM 
            pcr.setrandomseed(seed)
            timesteps = n_timesteps
            myModel = FoodEnvironment()
            dynFrw = pcrfw.DynamicFramework(myModel, timesteps)
            dynFrw.run()
                
            # Export infomation to gpkg files so we can visualise our data in QGIS
            dataset = ldm.open_dataset('food_environment.lue')
            for i in range(1, n_timesteps + 1):
                dataframe = campo.dataframe.select(dataset.hh, property_names=['bmi','income'])
                campo.to_gpkg(dataframe, 'households.gpkg', 'EPSG:28992', i)
            
            dataframe = campo.dataframe.select(dataset.hh, property_names=['bmi', 'income'])
            campo.to_csv(dataframe, "households.csv")

            # Getting agents district from the output
            df_bmi_hh = pd.read_csv("households_bmi.csv")
            df_bmi_loc = pd.read_csv("households_coords.csv")

            # Joining to get district of each household
            res = df_bmi_loc.merge(agents_per_district, how='inner', left_on=['CoordX', 'CoordY'], right_on=['x', 'y'])
            res.reset_index(inplace = True, drop = True)
            bmi_T = df_bmi_hh.T.reset_index()
            
            # df_bmi_hh.T (we need the tranpose of the df to get the results per agent)
            resu = pd.concat([res,bmi_T], axis = 1)
            resu.drop(["CoordX", "CoordY", "duplicate"], inplace = True, axis = 1)

            #Measure error
            #Excluding gap points
            #2014,2016,2018,2020 years at 5,11,17,23 columns
            lista_puntos_drop= [0,1,2,3,4,6,7,8,9,10,12,13,14,15,16,18,19,20,21,22]
            resu.drop(lista_puntos_drop, inplace = True, axis = 1)

            #Start checking overweight % per district
            resu.loc[resu["BMI"] >= 25  ,  "2012_over"] = 1
            resu.loc[resu["BMI"] < 25  ,  "2012_over"] = 0

            resu.loc[resu[5] >= 25  ,  "2014_over"] = 1
            resu.loc[resu[5] < 25  ,  "2014_over"] = 0

            resu.loc[resu[11] >= 25  ,  "2016_over"] = 1
            resu.loc[resu[11] < 25  ,  "2016_over"] = 0

            resu.loc[resu[17] >= 25  ,  "2018_over"] = 1
            resu.loc[resu[17] < 25  ,  "2018_over"] = 0

            resu.loc[resu[23] >= 25  ,  "2020_over"] = 1
            resu.loc[resu[23] < 25  ,  "2020_over"] = 0

            #Groupby district average
            resu_2 = resu.groupby("district").mean().reset_index()

            #Reading real data
            real = pd.read_csv("overweight_data.csv" ,sep = ';')
            #Combinining to compare
            combi = resu_2.merge(real, right_on= "Wijken", left_on ="district", how = "inner" )
            combi.drop("Wijken", inplace = True, axis = 1)
                
            for district in districts:
                comb = combi.copy()
                
                comb = comb.loc[comb["district"] == district]

                #Calculating mae for each year
                comb["2012_over"] = (comb["2012_over"]*100).round(2)
                comb["2014_over"] = (comb["2014_over"]*100).round(2)
                comb["2016_over"] = (comb["2016_over"]*100).round(2)
                comb["2018_over"] = (comb["2018_over"]*100).round(2)
                comb["2020_over"] = (comb["2020_over"]*100).round(2)

                comb["2014_error"] = abs(comb["2014_over"] - comb["overgewicht (inclusief obesitas), 18+ jaar|2014"] )
                comb["2016_error"] = abs(comb["2016_over"] - comb["overgewicht (inclusief obesitas), 18+ jaar|2016"] )
                comb["2018_error"] = abs(comb["2018_over"] - comb["overgewicht (inclusief obesitas), 18+ jaar|2018"] )
                comb["2020_error"] = abs(comb["2020_over"] - comb["overgewicht (inclusief obesitas), 18+ jaar|2020"] )
                
                #Just took last 2 time steps to really capture tendency
                comb["avg_error"] = (comb["2018_error"] + comb["2020_error"])/2
                
                
                #Storing results
                k1_storage.append(k1)
                k2_storage.append(k2)
                k3_storage.append(k3)
                mae_storage.append(comb.avg_error.mean())
                district_storage.append(district)
    
    
    #Creating df of all k1,k2,k3 runs in the districts       
    results = pd.DataFrame()
    results["k1"] = k1_storage
    results["k2"] = k2_storage
    results["k3"] = k3_storage
    results["mae"] = mae_storage
    results["district"] = district_storage

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [27]:
#Getting the lowest mae combination of k1,k2,k3 per district
results = results.loc[results.groupby('district').mae.idxmin()]
results.reset_index(inplace =True,drop=True)
#Saving to csv
results.to_csv("district_k.csv")